## [Fail] Main

- Traverse the BST, maintaining the index of where you're at

- At some point, you will hit p or q (since both exist in the pst and are not equal). Let's say (without loss of generality) that we hit `p` first
    - Record the index of `p`

- Continue traversing until we hit `q`
    - Record index of `q`

- Using the indices of `p` and `q`, find the indices of all their ancestors
    - while `index_p`, `index_p = (index_p-1)//2`, record in an array P
    - while `index_q`, `index_q = (index_q-1)//2`, record in an array Q
    - Both of these are sorted lists, sorted from biggest to smallest, so we can find the intersection of P and Q in $O(min(a,b))$ time
        - The worst case for `a` and `b` is $\log(N)$ assuming a binary search tree
        - So this intersection takes $O(\log(N))$ time and $O(\log(N))$ space

- Find the maximum of this intersection by traversing both arrays with one pointer each, and shifting the larger pointer rightwards every time the indices are not equal

- Once we find the first equal value, return that value as the answer

- The traversal step is the bottleneck, which runs in $O(N)$ time, while we need $O(\log(N))$ space to store the arrays

In [3]:
from collections import deque

# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Solution:
    def lowestCommonAncestor(self, root: TreeNode, p: TreeNode, q: TreeNode) -> TreeNode:
        queue = deque([root])
        p_index, q_index = None, None
        index=0
        tree_nodes = []

        while queue:
            curr = queue.popleft()
            if curr is None:
                index+=1
                continue
            
            tree_nodes.append(curr)
            if curr == p:
                p_index = index
            if curr == q:
                q_index = index
            if p_index and q_index:
                break
            
            if curr.left:
                queue.append(curr.left)
            else:
                queue.append(None)
            
            if curr.right:
                queue.append(curr.right)
            else:
                queue.append(None)
            index+=1
        
        p_ancestors, q_ancestors = [p_index], [q_index]
        while p_index > 0:
            p_index = (p_index-1) // 2
            p_ancestors.append(p_index)
        
        while q_index > 0:
            q_index = (q_index-1) // 2
            q_ancestors.append(q_index)

        curr_p, curr_q = 0,0
        while p_ancestors[curr_p] != q_ancestors[curr_q]:
            if p_ancestors[curr_p] > q_ancestors[curr_q]:
                curr_p += 1
            else:
                curr_q += 1
        return tree_nodes[p_ancestors[curr_p]]

In [4]:
one=TreeNode(6)
two=TreeNode(2)
three=TreeNode(8)
four=TreeNode(0)
five=TreeNode(4)
six=TreeNode(7)
seven=TreeNode(9)
eight=None
nine=None
ten=TreeNode(3)
eleven=TreeNode(5)

one.left=two
one.right=three
two.left=four
two.right=five
three.left=six
three.right=seven
four.left=eight
four.right=nine
five.left=ten
five.right=eleven

soln=Solution()
soln.lowestCommonAncestor(one, eleven, ten).val

4

In [13]:
# from binarytree import build

# # Example array
# arr = [
#     45,
#     30, 46,
#     10,36, None,49,
#     8,24, 34,42, 48,None, 4,9,
#     14,25, 31,35, 41,43, 47,None, 0,6, None,None, 11,20, None,28,
#     None,33, None,None, 37,None, None,44, None,None, None,1, 5,7, None,12, 19,21, 26,29, 32,None, None,38, None,None, None,3, None,None, None,None,
#     None,13, 18,None, None,22, None,27, None,None, None,None, None,39, 2,None, None,None, 15,None, None,23, None,None, None,40, None,None, None,16, None,None, None,None, None,17]

# # Build the binary tree from the array
# root = build(arr)

# # Print the binary tree
# print(root)

## Review

- This question is a good example of READING THE DAMN QUESTION PROPERLY
- Firstly, you were solving this as a BT, but the question tells you the structure is a BST. A BST has all left children strictly smaller, and right elements strictly larger
- The correct solution makes use of this info

- Also your solution works for the examples given, but I think one of the test cases in leetcode is broken, because it's not actually a valid BST, so this fails. 
    - The test case is given as `[45, 30, 46, 10, 36, null, 49, 8, 24, 34, 42, 48, null, 4, 9, 14, 25, 31, 35, 41, 43, 47, null, 0, 6, null, null, 11, 20, null, 28, null, 33, null, null, 37, null, null, 44, null, null, null, 1, 5, 7, null, 12, 19, 21, 26, 29, 32, null, null, 38, null, null, null, 3, null, null, null, null, null, 13, 18, null, null, 22, null, 27, null, null, null, null, null, 39, 2, null, null, null, 15, null, null, 23, null, null, null, 40, null, null, null, 16, null, null, null, null, null, 17]`
    - This is obviously problematic, because index 5 is null, but index 11 and 12 have values...